In [ ]:
# python 2/3 compatibility
from __future__ import print_function
# numerical python
import numpy as np
# file management tools
import glob
import os
# good module for timing tests
import time
# plotting stuff
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# ability to read/write fits files
from astropy.io import fits
# fancy image combination technique
from astropy.stats import sigma_clip
# median absolute deviation: for photometry
from astropy.stats import mad_std
# photometric utilities
from photutils import DAOStarFinder,aperture_photometry, CircularAperture, CircularAnnulus, Background2D, MedianBackground
# periodograms
from astropy.stats import LombScargle
from regions import read_ds9, write_ds9
from astropy.wcs import WCS

import numpy.ma as ma
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

np.set_printoptions(suppress=True)

In [ ]:
from DifferentialPhotometry_functions import *

## Initial setup: 

You will need to define the data directory where your images exist, a region file that you have created for the target members in your field, and a fits image with WCS information. 

Also, define a signal to noise threshold for detrending the photometry.  If detrending fails, lower the threshold. 

In [ ]:
# name of field
target = "praesepe"
filt = 'V' 

#location of data
datadir = '/Volumes/ARCTURUS/WIYN09/SmallStack_' + target + '/' + filt + '/'

#grab data
im = glob.glob(datadir + '*N*.fits')

#S/N threshold
sn_thresh=3

# For your own data, modify as needed
wcs_image = '/Volumes/ARCTURUS/WIYN09/RegionFiles/' + target+ '/' + target + '_WCS.fits'

#optional region file 
wcs_region = '/Volumes/ARCTURUS/WIYN09/RegionFiles/' + target + '/' + target + '_VZ_sm.reg'

In [ ]:
print(f"Number of images found: {len(im)}")
print(np.array(im))

In [ ]:
aprad=20.  # aperture radius for HDI
skybuff=14.  # sky annulus inner radius
skywidth=18.  # sky annulus outer radius

In [ ]:
# sensitivities for star finding
nsigma=4.5 # detection threshold in sigma 
FWHM=5. # pixels 

In [ ]:
## do starfind on one of the images.
xpos, ypos, nstars = StarFind(im[2], FWHM, nsigma)

In [ ]:
#find ra, dec coordinates of stars
hdr_wcs = fits.getheader(wcs_image)
w = construct_astrometry(hdr_wcs)
ra, dec = w.wcs_pix2world(xpos, ypos,1)

In [ ]:
# Note that you will need to update the "timekey" keyword to pick the right header keyword from AIJ! 

times, Photometry_initial = doPhotometry(im, xpos, ypos,aprad, skybuff, skywidth)

In [ ]:
ePhotometry = doPhotometryError(im,xpos, ypos, aprad, skybuff, skywidth, Photometry_initial, manual=True, xboxcorner=2000, yboxcorner=2000, boxsize=200)

In [ ]:
#find ra, dec coordinates of stars

#using region file
memberlist = wcs_region

#manual inputting 1 star
# ra_in, dec_in = w.wcs_pix2world([2402],[2076],1)
# memberlist = (ra_in, dec_in)

#determine index, ra, dec of target stars from the DAOstarfinder catalogue.
idx, RA, DEC = target_list(memberlist, ra, dec)

In [ ]:
print('indices of targets in starfinder catalogue: {}'.format(idx) )
print('number of target stars found: {}'.format(len(idx)))

In [ ]:
#detrend photometry and plot 
Photometry, cPhotometry = detrend(idx, Photometry_initial, ePhotometry, nstars, sn_thresh)
plotPhotometry(times,cPhotometry)

In [ ]:
# determine most accurate comparison stars 
most_accurate = findComparisonStars(Photometry, cPhotometry) # comp_num= (can set this)

In [ ]:
#run differential photometry on all stars
dPhotometry, edPhotometry, tePhotometry  = runDifferentialPhotometry(Photometry_initial, ePhotometry, nstars, most_accurate, sn_thresh=2)

In [ ]:
#pull out differential photometry for target stars and save to .npz file
tar_ra, tar_dec, tar_xpos, tar_ypos, time, flux, fluxerr, foldedphase, totperiod, totpower  = diffPhot_IndividualStars(datadir, idx, ra, dec, xpos, ypos, dPhotometry, edPhotometry, tePhotometry,times, target, filt, wcs_image, most_accurate)
#


In [ ]:
#open data
data = np.load(datadir + 'differentialPhot_field' + target + filt + '.npz')

#read column headers
data.files

print(len(data['ra']))
#access data with data['flux']

# Plot the lightcurve here!


In [ ]:
for star in np.arange(len(data['flux'])):

    fig, (ax0, ax1, ax2) = plt.subplots(figsize=(15,4), nrows=1, ncols=3)
    time = data['time']
    ax0.errorbar(data['time'], data['flux'][star], yerr=data['fluxerr'][star], fmt='ko')

    ax1.plot(data['period'][star], data['power'][star], 'k')

    ax2.plot(data['foldedphase'][star]-1., data['flux'][star], 'o', color='gray')
    ax2.plot(data['foldedphase'][star], data['flux'][star], 'ko')
    ax2.plot(data['foldedphase'][star]+1., data['flux'][star], 'o', color='gray')
    
    ax0.set_xlabel('time [day]')
    ax0.set_ylabel('relative flux')
    ax1.set_xlabel('period [days]')
    ax1.set_ylabel('power')
    ax2.set_xlabel('phase')
    ax2.set_ylabel('relative flux')
    
#     ax0.set_ylim(0.5, 1.5)

#     print('flux:', data2['flux'][star])
#     print('phase:',data2['foldedphase'][star])
#     print('fluxerr:',data2['fluxerr'][star])
#     print('-------------')
    plt.suptitle('star at: (x, y) = (' + str(data['xpos'][star]) + ', ' + str(data['ypos'][star]) + ')' )
       
plt.show()